# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,gadzhiyevo,69.2551,33.3362,-16.07,96,100,3.27,RU,1705400928
1,1,sao joao da barra,-21.6403,-41.0511,27.05,87,3,5.10,BR,1705400877
2,2,ketchikan,55.3422,-131.6461,0.26,93,100,1.54,US,1705401153
3,3,tsubata,36.6667,136.7333,2.32,83,99,2.31,JP,1705401153
4,4,jayapura,-2.5333,140.7000,24.66,93,100,1.19,ID,1705401153


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points( 
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City" 
)

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_1 = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27)]
city_data_2 = city_data_1.loc[city_data_1['Wind Speed'] < 4.5]
city_data_3 = city_data_2.loc[city_data_2['Cloudiness'] == 0]

# Drop any rows with null values
city_data_complete = city_data_3.dropna(how='any')

# Display sample data
city_data_complete

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
98,98,arraial do cabo,-22.9661,-42.0278,26.92,78,0,4.12,BR,1705400943
159,159,tamanrasset,22.7850,5.5228,21.95,5,0,1.03,DZ,1705401190
167,167,pedernales,18.0384,-71.7440,24.35,67,0,1.05,DO,1705401192
215,215,rio grande,-32.0350,-52.0986,24.06,88,0,3.34,BR,1705400924
220,220,nawabganj,24.5911,88.2710,21.87,38,0,1.37,BD,1705401206
222,222,palpala,-24.2565,-65.2116,22.57,88,0,2.57,AR,1705401206
313,313,guaiba,-30.1139,-51.3250,25.16,89,0,2.57,BR,1705401227
430,430,terenos,-20.4422,-54.8603,25.84,78,0,3.09,BR,1705401256
479,479,cidreira,-30.1811,-50.2056,25.32,75,0,2.59,BR,1705401267
545,545,palma soriano,20.2172,-75.9989,21.48,92,0,0.88,CU,1705401284


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_complete[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
98,arraial do cabo,BR,-22.9661,-42.0278,78,
159,tamanrasset,DZ,22.7850,5.5228,5,
167,pedernales,DO,18.0384,-71.7440,67,
215,rio grande,BR,-32.0350,-52.0986,88,
220,nawabganj,BD,24.5911,88.2710,38,
222,palpala,AR,-24.2565,-65.2116,88,
313,guaiba,BR,-30.1139,-51.3250,89,
430,terenos,BR,-20.4422,-54.8603,78,
479,cidreira,BR,-30.1811,-50.2056,75,
545,palma soriano,CU,20.2172,-75.9989,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
arraial do cabo - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
pedernales - nearest hotel: Hostal Doña Chava
rio grande - nearest hotel: Hotel Vila Moura Executivo
nawabganj - nearest hotel: Hotel Al Nahid International হোটেল আল নাহিদ আন্তর্জাতিক
palpala - nearest hotel: Hotel Municipal ex Casino
guaiba - nearest hotel: Ibis Guaíba
terenos - nearest hotel: No hotel found
cidreira - nearest hotel: Hotel Castelo
palma soriano - nearest hotel: Hotel Palma


,City,Country,Lat,Lng,Humidity,Hotel Name
98,arraial do cabo,BR,-22.9661,-42.0278,78,No hotel found
159,tamanrasset,DZ,22.7850,5.5228,5,فندق الأمان
167,pedernales,DO,18.0384,-71.7440,67,Hostal Doña Chava
215,rio grande,BR,-32.0350,-52.0986,88,Hotel Vila Moura Executivo
220,nawabganj,BD,24.5911,88.2710,38,Hotel Al Nahid International হোটেল আল নাহিদ আন...
222,palpala,AR,-24.2565,-65.2116,88,Hotel Municipal ex Casino
313,guaiba,BR,-30.1139,-51.3250,89,Ibis Guaíba
430,terenos,BR,-20.4422,-54.8603,78,No hotel found
479,cidreira,BR,-30.1811,-50.2056,75,Hotel Castelo
545,palma soriano,CU,20.2172,-75.9989,92,Hotel Palma


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points( 
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.2,
    color = "City",
    hover_cols=[ "Country", "Hotel Name"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)